In [1]:
import pandas as pd

data = pd.read_csv("D:/Data Mining/HW2/Reviews.csv", nrows = 10000)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Id                      10000 non-null  int64 
 1   ProductId               10000 non-null  object
 2   UserId                  10000 non-null  object
 3   ProfileName             10000 non-null  object
 4   HelpfulnessNumerator    10000 non-null  int64 
 5   HelpfulnessDenominator  10000 non-null  int64 
 6   Score                   10000 non-null  int64 
 7   Time                    10000 non-null  int64 
 8   Summary                 10000 non-null  object
 9   Text                    10000 non-null  object
dtypes: int64(5), object(5)
memory usage: 781.4+ KB


In [2]:
data = data.loc[:, ["Score", "Text"]]

score_4 = []
for row in data.itertuples():
    if row.Score >= 4:
        score_4.append(1)
    else:
        score_4.append(0)
data["Score"] = score_4
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Score   10000 non-null  int64 
 1   Text    10000 non-null  object
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


In [3]:
from nltk.corpus import stopwords

data["New_Text"] = data["Text"].str.lower()
sw = set(stopwords.words("english"))

data["New_Text"] = data["New_Text"].apply(lambda x: " ".join([word for word in x.split() if word not in sw]))
print(data["New_Text"][:5])

0    bought several vitality canned dog food produc...
1    product arrived labeled jumbo salted peanuts.....
2    confection around centuries. light, pillowy ci...
3    looking secret ingredient robitussin believe f...
4    great taffy great price. wide assortment yummy...
Name: New_Text, dtype: object


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfid = TfidfVectorizer()
tfid = tfid.fit_transform(data["New_Text"])

print(tfid.shape)

df1_A = data.loc[:, ["Score"]]
df1_B = pd.DataFrame(tfid.toarray())
df1 = pd.concat([df1_A, df1_B], axis = 1)
print(df1[:5])

(10000, 18792)
   Score    0    1    2    3    4    5    6    7    8  ...  18782  18783  \
0      1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
1      0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
2      1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
3      0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
4      1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   

   18784  18785  18786  18787  18788  18789  18790  18791  
0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
1    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
2    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
3    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
4    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  

[5 rows x 18793 columns]


In [5]:
import gensim
from gensim.models.word2vec import Word2Vec

tokenized_text = data["New_Text"].apply(lambda x: x.split())
model_w2v = gensim.models.Word2Vec(
            tokenized_text,
            window = 5,
            min_count = 2,
            sg = 1,
            hs = 0,
            negative = 10,
            workers = 32,
            seed = 34)



In [6]:
model_w2v.build_vocab(tokenized_text, progress_per = 1000)

In [7]:
model_w2v.train(tokenized_text, total_examples = model_w2v.corpus_count, epochs = model_w2v.epochs)

(1835632, 2047285)

In [8]:
model_w2v.save("D:/Data Mining/HW2/W2V")

In [9]:
model_w2v.wv.most_similar("good")

[('decent', 0.7414308786392212),
 ('fantastic', 0.7252123951911926),
 ('good,', 0.6955394148826599),
 ('greatest', 0.6875644326210022),
 ('great', 0.6864476203918457),
 ('compromise', 0.6838355660438538),
 ('mediocre', 0.6828298568725586),
 ('terrific', 0.6761401295661926),
 ('affect', 0.6712435483932495),
 ('compares', 0.6711976528167725)]

In [10]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_w2v.wv[word].reshape((1, size))
            count += 1
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec
                

In [11]:
import numpy as np
w2v_arrays = np.zeros((len(tokenized_text), 100))
for i in range(len(tokenized_text)):
    w2v_arrays[i,:] = word_vector(tokenized_text[i], 100)
w2v_df = pd.DataFrame(w2v_arrays)

print(w2v_df.shape)

(10000, 100)


In [12]:
df2_A = data.loc[:, ["Score"]]
df2_B = w2v_df
df2 = pd.concat([df2_A, df2_B], axis = 1)
print(df2[:5])

   Score         0         1         2         3         4         5  \
0      1 -0.195535 -0.395347  0.001792  0.138430 -0.178449 -0.261397   
1      0 -0.339054 -0.389655  0.019959  0.024976 -0.073523  0.062920   
2      1 -0.162290 -0.414272  0.069906  0.081594 -0.218892  0.134486   
3      0 -0.182823 -0.217439 -0.064454  0.131533 -0.125639  0.206125   
4      1 -0.155265 -0.319211 -0.087308  0.160939 -0.138497  0.047058   

          6         7         8  ...        90        91        92        93  \
0 -0.119046 -0.213248  0.378351  ...  0.148595 -0.509286  0.270203  0.180871   
1 -0.031040 -0.151524  0.289774  ...  0.263520 -0.430171  0.189792  0.255900   
2 -0.042746 -0.280327  0.378188  ...  0.195344 -0.327714  0.161781  0.157842   
3  0.125657 -0.261090  0.403230  ...  0.152565 -0.413069  0.212136  0.030154   
4 -0.117555 -0.329421  0.391538  ...  0.252161 -0.363431  0.137474  0.377906   

         94        95        96        97        98        99  
0  0.251244 -0.288579 

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from pandas import DataFrame
from decimal import *

def K_fold_CV(k, data):
    subset_size = int(data.shape[0] / k)
    accuracy = Decimal(0)
    
    for i in range(k):
        train_1 = data.iloc[:i * subset_size]
        train_2 = data.iloc[i * subset_size + subset_size:len(data)]
        train = pd.concat([train_1, train_2])
        test = data.iloc[i * subset_size: i * subset_size + subset_size]
        
        X_train = train.iloc[:, 1:]
        Y_train = train["Score"]
        X_test = test.iloc[:, 1:]
        Y_test = test["Score"]
        
    
        rfc = RandomForestClassifier(max_depth=5, random_state=0)
        rfc.fit(X_train, Y_train)
        predict = rfc.predict(X_test)
        accuracy += Decimal(accuracy_score(Y_test, predict))
    
    return accuracy / k


In [14]:
accuracy = K_fold_CV(4, df1)
print(accuracy)

0.761599999999999999200639422


In [15]:
accuracy = K_fold_CV(4, df2)
print(accuracy)

0.7647000000000000186073378928
